In [1]:
from jetbot import Camera
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg',width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [2]:
import os

red_dir = 'dataset/red'
orange_dir = 'dataset/orange'
green_dir = 'dataset/green'

try :
    os.makedirs(red_dir)
    os.makedirs(orange_dir)
    os.makedirs(green_dir)
except FileExistsError:
    print("Directories not created because they already exist")

In [4]:
button_layout = widgets.Layout(width='128px', height='64px')
red_button = widgets.Button(description='add RED', button_style='success'
                           , layout = button_layout)
red_button.style.button_color = 'red'

orange_button = widgets.Button(description='add ORANGE', button_style='success'
                           , layout = button_layout)
orange_button.style.button_color = 'orange'

green_button = widgets.Button(description='add GREEN', button_style='success'
                           , layout = button_layout)
green_button.style.button_color = 'green'


red_count = widgets.IntText(layout=button_layout, value=len(os.listdir(red_dir)))
orange_count = widgets.IntText(layout=button_layout, value=len(os.listdir(orange_dir)))
green_count = widgets.IntText(layout=button_layout, value=len(os.listdir(green_dir)))

display(widgets.HBox([red_count, red_button]))
display(widgets.HBox([orange_count, orange_button]))
display(widgets.HBox([green_count, green_button]))

In [5]:
from uuid import uuid1

def save_snapshot(directory) :
    image_path = os.path.join(directory, str(uuid()) + '.jpg')
    with open(image_path, 'wb') as f :
        f.write(image.value)
    
def save_red() :
    global red_dir, red_count
    save_snapshot(red_dir)
    red_count.value = len(os.listdir(red_dir))
    
def save_orange() :
    global orange_dir, orange_count
    save_snapshot(orange_dir)
    orange_count.value = len(os.listdir(orange_dir))
    
def save_green() :
    global green_dir, green_count
    save_snapshot(green_dir)
    green_count.value = len(os.listdir(green_dir))
    

    
red_button.on_click(lambda x: save_red())
orange_button.on_click(lambda x: save_orange())
green_button.on_click(lambda x: save_green())

In [6]:
display(image)
display(widgets.HBox([red_count, red_button]))
display(widgets.HBox([orange_count, orange_button]))
display(widgets.HBox([green_count, green_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [7]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1,0.1,0.1,0.1),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

In [ ]:
model = models.alexnet(pretrained=True)

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.901, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        
    test_error_count = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy
        

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value) : 
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2,0,1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=5)
image = widgets.Image(format='jpeg',width=224, height=224)
red_slider = widgets.FloatSlider(description='red', min=0.0, max=1.0, orientation='vertical')
orange_slider = widgets.FloatSlider(description='orange', min=0.0, max=1.0, orientation='vertical')
green_slider = widgets.FloatSlider(description='green', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image,red_slider, orange_slider, green_slider]))

In [ ]:
from jetbot import Robot
import time

robot = Robot()

In [ ]:
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()
RGB.Set_WaterfallLight_RGB()
RGB.Set_ChameleonLight_RGB()
RGB.Set_BreathColor_RGB()
RGB.Set_BreathSColor_RGB(0)
RGB.Set_BreathSSpeed_RGB(3)
RGB.Set_BreathSLight_RGB()
RGB.OFF_ALL_RGB()

In [ ]:
import torch.nn.functional as F
import time

def update(change) :
    global blocked_slider, robot
    x = change['new']
    x = preprocess(x)
    y = model(x)
    
    y = F.softmax(y, dim=1)
    
    prob_red = float(y.flatten()[2])
    prob_orange = float(y.flatten()[1])
    prob_green = float(y.flatten()[0])
    
    red_slider.value = prob_red
    orange_slider.value = prob_orange
    green_slider.value = prob_green
    
    if prob_red > 0.6 :
        robot.forward(0.0)
        time.sleep(0.1)
        RGB.Set_All_RGB(0xFF, 0x00, 0x00)
        time.sleep(0.5)

    else :
        robot.forward(0.3)
        
    if prob_orange > 0.6 :
        robot.forward(0.15)
        time.sleep(0.1)
    else :
        robot.forward(0.3)
        
    if prob_green > 0.6 :
        robot.forward(0.3)
        RGB.Set_All_RGB(0x00, 0xFF, 0x00)
        time.sleep(0.5)
        
    time.sleep(0.001)

update({'new' : camera.value})